# SphericalInvariants example

To install rascal:
(NOTE: See the top-level README for the most up-to-date installation instructions.)
+ mkdir ../build 
+ cd build
+ cmake -DCMAKE_BUILD_TYPE=Release -DBUILD_TESTS=ON ..
+ make -j 4
+ make install

In [ ]:
%env OMP_NUM_THREADS=1

%matplotlib inline
from matplotlib import pyplot as plt

import os, sys
from ase.io import read
sys.path.insert(0,"../build/")

import sys
import time
import rascal
import json

import ase
from ase.io import read
from ase.build import make_supercell
from ase.visualize import view
import numpy as np
import sys

from copy import deepcopy,copy

import json

from rascal.representations import SphericalExpansion
from rascal.utils import ostream_redirect

In [ ]:
frames = ase.io.read('../tests/reference_data/dft-smiles_500.xyz',index=':10')
new_frames = []
aa = []
for frame in frames:
    new_frames.append(make_supercell(frame,np.eye(3)*1))
print(np.max(list(map(len,new_frames))))

In [ ]:
hypers = dict(interaction_cutoff=6, 
              max_radial=6, 
              max_angular=3, 
              gaussian_sigma_constant=0.5,
              gaussian_sigma_type="Constant",
              cutoff_smooth_width=0.5,
              )
sph_expn = rascal.representations.SphericalExpansion(**hypers)

In [ ]:
with ostream_redirect():
    expansions = sph_expn.transform(new_frames)

In [ ]:
X = expansions.get_dense_feature_matrix(sph_expn)

In [ ]:
X.shape

In [ ]:
%timeit -n 2 -r 1 expansions = sph_expn_test.transform(new_frames)

In [ ]:
%timeit -n 2 -r 1 expansions = sph_expn.transform(new_frames)

In [ ]:
%timeit -n 2 -r 1 expansions = sph_expn_test.transform(new_frames)

In [ ]:
expansions.get_dense_feature_matrix()

Good, should be 5 centres (5 atoms) * 10 radial * 81 ${}=(l+1)^2$